In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import geopandas as gpd

In [ ]:
#importing all 4 datasets

taxi_df = pd.read_csv("Data\\yellow_tripdata_2023-01.csv", parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
holiday_df = pd.read_excel("Data\\us_holidays_2023.xlsx", parse_dates=[0], names=['date', 'day_of_week', 'holiday_name', 'holiday_type'])
weather_df = pd.read_csv("Data\\newyork 2023-01-01 to 2023-01-31.csv", parse_dates=['datetime'])

gdf = gpd.read_file("Data\\NYC Taxi Zones.geojson")
taxi_zone = gdf.explode(index_parts=False).reset_index(drop=True)

In [ ]:
taxi_df.shape

In [ ]:
taxi_df.head()

In [ ]:
print("taxi_df")
print(taxi_df.columns)
print("holiday_df")
print(holiday_df.columns)
print("weather_df")
print(weather_df.columns)
print("taxi_zone")
print(taxi_zone.columns)

In [ ]:
#data cleaning

taxi_df['tpep_pickup_datetime'] = pd.to_datetime(taxi_df['tpep_pickup_datetime'])

taxi_df = taxi_df[
    (taxi_df['trip_distance'] > 0) &
    (taxi_df['PULocationID'].notnull()) &
    (taxi_df['DOLocationID'].notnull()) &
    (taxi_df['tpep_pickup_datetime'].dt.year == 2023) &
    (taxi_df['tpep_pickup_datetime'].dt.month == 1)
]


In [ ]:
taxi_df.head()

In [ ]:
holiday_df.head()

In [ ]:
holiday_df['date'] = pd.to_datetime(holiday_df['date']).apply(lambda x: x.replace(year=2023))

In [ ]:
weather_df.head(5)

In [ ]:
taxi_zone.head(5)

In [ ]:
#handling null values

taxi_df['passenger_count'].fillna(taxi_df['passenger_count'].mode()[0],inplace=True)
taxi_df['RatecodeID'].fillna(taxi_df['RatecodeID'].mode()[0],inplace=True)
taxi_df['store_and_fwd_flag'].fillna('N',inplace=True)
taxi_df['congestion_surcharge'].fillna(0, inplace=True)
taxi_df['airport_fee'].fillna(0, inplace=True)
taxi_df.isna().sum()

In [ ]:
#selecting necessary features and Feature Engineering

taxi_df = taxi_df[[
    'tpep_pickup_datetime', 'PULocationID', 'trip_distance'
]]
taxi_df = taxi_df.dropna()
taxi_df['pickup_hour'] = taxi_df['tpep_pickup_datetime'].dt.hour
taxi_df['pickup_date'] = taxi_df['tpep_pickup_datetime'].dt.date


In [ ]:
holiday_df['date'] = pd.to_datetime(holiday_df['date'])

In [ ]:
weather_df = weather_df[[
    'datetime', 'temp', 'humidity', 'precip', 'windgust', 'windspeed', 'visibility', 'conditions'
]]
weather_df['hour'] = weather_df['datetime'].dt.hour
weather_df['date'] = weather_df['datetime'].dt.date

In [ ]:
holiday_df.isna().sum()

In [ ]:
taxi_zone = taxi_zone.rename(columns={'location_id': 'PULocationID'})  # For merge
taxi_zone = taxi_zone[['PULocationID', 'zone', 'borough', 'geometry']]


In [ ]:
df_demand = taxi_df.groupby(['pickup_date', 'pickup_hour', 'PULocationID']).agg(
    num_trips=('PULocationID', 'count'),
    avg_trip_distance=('trip_distance', 'mean')
).reset_index()

In [ ]:
weather_df.isna().sum()

In [ ]:
df_demand['pickup_date'] = pd.to_datetime(df_demand['pickup_date'])
df_demand['pickup_date'].info()

In [ ]:
weather_df['datetime'].info()

In [ ]:
df_demand.head(5)

In [ ]:
weather_df.head(5)

In [ ]:
#merge with weather
df_merged = df_demand.merge(weather_df, how='left', left_on = 'pickup_date',right_on = 'datetime')

In [ ]:
df_merged.drop(columns=['date', 'datetime', 'hour'], inplace=True)

In [ ]:
df_merged.isnull().sum()

In [ ]:
holiday_df['date'] = holiday_df['date'].apply(lambda x: x.replace(year=2023))

In [ ]:
df_merged.head(5)

In [ ]:
holiday_df.head(5)

In [ ]:
#merge with holiday

df_merged = df_merged.merge(holiday_df[['date','holiday_name']], how = 'left', left_on = 'pickup_date', right_on = 'date')
df_merged['is_holiday'] = df_merged['holiday_name'].notna().astype(int)

In [ ]:
df_merged.head(5)

In [ ]:
df_merged[~df_merged['pickup_date'].isin(holiday_df['date'])].shape


In [ ]:
df_merged['holiday_name'].fillna('No Holiday', inplace=True)

In [ ]:
df_merged = df_merged.drop(columns=['date'],axis=1)

In [ ]:
taxi_zone['PULocationID'] = taxi_zone['PULocationID'].astype('int64')
df_merged = df_merged.merge(taxi_zone, how='left', on='PULocationID')


In [ ]:
df_merged.isnull().sum()

In [ ]:
df_output = df_merged[[
    'pickup_hour', 'pickup_date','PULocationID', 'zone', 'borough', 'num_trips',
    'temp', 'humidity', 'precip', 'windgust', 'windspeed',
    'visibility', 'conditions', 'is_holiday'
]]

In [ ]:
df_output.dropna(inplace=True)

In [ ]:
df_output.isnull().sum()

In [ ]:
df_output.head(15)

In [ ]:
df_output.drop_duplicates(inplace=True)

In [ ]:
import folium
from folium.plugins import HeatMap

In [ ]:
# Aggregate demand by zone (PULocationID)
zone_demand = df_output.groupby(['PULocationID','zone','borough']).agg({'num_trips': 'sum'}).reset_index()

# Merge with taxi_zone GeoDataFrame for geometry
zone_geo = taxi_zone.merge(zone_demand, on='PULocationID')

In [ ]:
zone_geo = zone_geo.drop(columns=['zone_x','borough_y'])
zone_geo = zone_geo.rename(columns={'zone_z': 'zone','borough_x':'borough'})

In [ ]:
# Convert to GeoJSON for Folium

zone_geo = gpd.GeoDataFrame(zone_geo, geometry='geometry')

In [ ]:
# Create base map

m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

In [ ]:
# Add Choropleth for demand

folium.Choropleth(
    geo_data=zone_geo,
    data=zone_geo,
    columns=['PULocationID', 'num_trips'],
    key_on='feature.properties.PULocationID',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Total Trips'
).add_to(m)

In [ ]:
# Display the map

m.save("zone_demand_heatmap.html")

In [ ]:
zone_geo['centroid'] = zone_geo.geometry.centroid
zone_geo['lat'] = zone_geo.centroid.y
zone_geo['lon'] = zone_geo.centroid.x

heat_data = list(zip(zone_geo['lat'],zone_geo['lon'],zone_demand['num_trips']))

m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)
HeatMap(heat_data, radius=15).add_to(m)
m.save('heatmap1.html')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

hourly_demand = df_output.groupby('pickup_hour')['num_trips'].mean()

plt.figure(figsize=(12,6))
sns.lineplot(data=hourly_demand)
plt.title('Average Demand by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Number of Trips')
plt.xticks(range(0,24))
plt.show()

In [ ]:
df_output['weekday'] = pd.to_datetime(df_output['pickup_date']).dt.dayofweek
df_output['is_weekend'] = df_output['weekday'].apply(lambda x: 1 if x>=5 else 0)

In [ ]:
weekend_demand = df_output.groupby(['pickup_hour','is_weekend'])['num_trips'].mean().reset_index()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,6))

# Create the plot with custom colors
sns.lineplot(data=weekend_demand, x='pickup_hour', y='num_trips', hue='is_weekend', 
             palette={False: '#1f77b4', True: '#ff7f0e'})  # Blue for weekday, orange for weekend

plt.title('Demand by Hour: Weekday vs Weekend')
plt.xlabel('Hour of Day')
plt.ylabel('Average Number of Trips')
plt.xticks(range(0,24))

# Fix the legend
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles, ['Weekday', 'Weekend'])

plt.show()

In [ ]:
rainy = df_output[df_output['precip'] > 0]
clear = df_output[df_output['precip'] == 0]

print(f"Average demand on rainy days: {rainy['num_trips'].mean()}")
print(f"Average demand on clear days: {clear['num_trips'].mean()}")

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

df_model = df_output.copy()

# Label encode categorical columns
for col in ['zone','borough','conditions']:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col].astype(str))

In [ ]:
df_model['day_of_week'] = pd.to_datetime(df_model['pickup_date']).dt.dayofweek

In [ ]:
# Lag feature (previous hour demand per zone)
df_model = df_model.sort_values(['PULocationID', 'pickup_date', 'pickup_hour'])
df_model['lag_1'] = df_model.groupby('PULocationID')['num_trips'].shift(1).fillna(0)


In [ ]:
# Target
y = df_model['num_trips']
X = df_model.drop(columns=['pickup_date', 'num_trips'])

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Test RMSE: {rmse}")

In [ ]:
df_output.to_csv("urban_mobility_demand_forecast.csv", index=False)